<div style="border: solid 1px #3b8bed; padding: 5px; border-radius: 8px; margin-top: 8px">

**All my answers/comments are in blue boxes**
</div>

**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd, adam
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization

Using TensorFlow backend.


# MiniProject on Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [2]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [3]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self, e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

<div style="border: solid 1px #3b8bed; padding: 5px; border-radius: 8px; margin-top: 8px">

When `train` is `True`, the function randomly selects one of the two behaviour : 
- **Exploration** *(with probability `epsilon`)*: the Agent randomly selects an action
- **Exploitation** *(with probability $1 - $`epsilon`)*: the Agent selects the best action according to what it learns until now. 

Thus, `epsilon` is essential because it controls the trade-off between exploration and exploitation



When `train` is `False`, exploitation is always chosen.

</div>

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

<div style="border: solid 1px #3b8bed; padding: 5px; border-radius: 8px; margin-top: 8px">

I re-factored the `Environment` class so it will be easy to write `EnvironmentExploring`as a sub-class of `Environment`. I did this to avoid too much code redundancy.

</div>

In [4]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size, self.grid_size, 3)) + 128
        b[self.board > 0, 0] = 256
        b[self.board < 0, 2] = 256
        b[self.x, self.y, :]=256
        b[-2:, :, :]=0
        b[: ,-2:, :]=0
        b[:2, :, :]=0
        b[:, :2, :]=0
        
        b = cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t, :, :, :] = b
    
    def create_position(self, x=None, y=None):
        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        if(x is not None):
            self.position[x, y] = 1
            
    def move(self, action):
        if action == 0:
            if self.x == self.grid_size - 3:
                self.x = self.x - 1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x + 1
            else:
                self.x = self.x - 1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')
            
        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        return reward, game_over

    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""
        self.get_frame(int(self.t))
        
        self.create_position(self.x, self.y)
        reward, game_over = self.move(action)
        state = self.create_state(self.board, self.position)
        return state, reward, game_over

    def create_state(self, *state_grids):
        state = np.concatenate([
            state_grid.reshape(self.grid_size, self.grid_size,1) 
            for state_grid in state_grids
        ], axis=2)
        return state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
    
    def reset_board(self):
        """This function resets the board and the agent coordinates"""
        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]
        
        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))

        malus[bonus>0]=0

        self.board = bonus + malus
        self.board[self.x,self.y] = 0
    
    def reset(self):
        """This function resets the game and returns the initial state"""
        self.reset_board()
        self.create_position()
        self.t = 0
        
        return self.create_state(self.board, self.position)

The following elements are important because they correspond to the hyper parameters for this project:

In [5]:
# parameters
size = 13
T=200
temperature = 0.3
epochs_train = 5 # set small when debugging
epochs_test = 2 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

<div style="border: solid 1px #3b8bed; padding: 5px; border-radius: 8px; margin-top: 8px">

`position` is a grid with 0 everywhere but -1 on the borders (and 1 where the agent was on the previous turn). I think its purpose is to let the agent understand its position (so it can avoid colliding the borders for example).

`board` is a grid of the same size. It contains the rewards of each cell (0 -> nothing, 0.5 -> cheese, -1 -> poison).

</div>

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [6]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass
    
    def learned_act(self, s):
        return np.random.randint(0, 4)

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [7]:
def test(agent, env, epochs, prefix='', verbose=1):
    
    score = 0
    
    for e in range(1, epochs+1):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0
        
        while not game_over:
            # The agent performs an action
            action = agent.act(state, train=False)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward
        
        # Update stats
        score += win - lose
        
        if(e % verbose == 0):
            env.draw(prefix+str(e))
            print("Win/lose count {}/{}. Average score ({})"
                  .format(win, lose, score/e))
    print('Final score: '+str(score/epochs))

In [8]:
# Initialize the game
env = Environment(grid_size=size, max_time=T, temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

epochs_test = 50
test(agent, env, epochs_test, prefix='random', verbose=10)
HTML(display_videos(f'random{epochs_test}.mp4'))

Win/lose count 7.0/9.0. Average score (-5.0)
Win/lose count 12.5/6.0. Average score (-3.425)
Win/lose count 9.0/8.0. Average score (-3.9166666666666665)
Win/lose count 15.0/17.0. Average score (-3.875)
Win/lose count 15.5/11.0. Average score (-4.37)
Final score: -4.37


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p^{\pi}(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim p(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim p(.|s,a)}\Vert r+\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




<div style="border: solid 1px #3b8bed; padding: 5px; border-radius: 8px; margin-top: 8px">

We have that : 

$Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t=0}^{+\infty}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] $

$Q^\pi(s,a)=E_{p^{\pi}}[r(s, a) + \gamma \sum_{t=0}^{+\infty}\gamma^{t}r(s_{t+1},a_{t+1})|s_{0}=s,a_{0}=a] $

$Q^\pi(s,a)=E_{(s',a')\sim p^{\pi}(.|s,a)}[r(s, a) + \gamma E_{p^{\pi}}[ \sum_{t=0}^{+\infty}\gamma^{t}r(s_{t+1},a_{t+1})|s_{1}=s',a_{1}=a']] $

$Q^{\pi}(s,a)=E_{(s',a')\sim p^{\pi}(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]$

***

The existence of $\pi^*$, a policy that maximises $Q$ for all $(s, a)$ implies that $\max_{\pi} E_{s, a} [Q^\pi(s, a)] = E_{s, a}[Q^*(s, a)]$. <br>Indeed, by definition of $Q^*$, $\max_{\pi} E_{s, a} [Q^\pi(s, a)] \leq E_{s, a}[Q^*(s, a)]$. And by taking $\pi = \pi^*$, we get $E_{s, a} [Q^\pi(s, a)] = E_{s, a}[Q^*(s, a)]$.

Using this remark : 

$$
\begin{aligned}
 Q^*(s, a) & = \max_\pi Q^\pi(s, a) \\
 & = \max_{\pi = (\pi_0, \pi_1, \pi')} E_{s' \sim p(.|s, a), a'\sim \pi_1(s')}\left[r(s, a) + \gamma Q^{\pi'}(s', a')\right] \\
 & = E[r(s, a)] + \gamma \max_{\pi_1, \pi'} E_{s' \sim p(.|s, a), a'\sim \pi_1(s')}\left[ Q^{\pi'}(s', a')\right] \\
 & = E[r(s, a)] + \gamma \max_{\pi_1} E_{s' \sim p(.|s, a), a'\sim \pi_1(s')}\left[ Q^{*}(s', a')\right] & 
 \text{because } \pi^* \text{ exists}\\
 & = E[r(s, a)] + \gamma \max_{a'} E_{s' \sim p(.|s, a)}\left[ Q^{*}(s', a')\right] \\
 & = E_{s'\sim p(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{aligned}
$$

***

Let $\theta^*$ be an optimal parameter ($Q* = Q(., ., \theta^*)$. Then $\mathcal{L}(\theta^*) = 0$.

Plus, if we let $Q'$ be such that $\forall s,a, \, Q'(s, a) = E_{s'\sim p(.|s,a)}[r(s,a)+\gamma\max_{a'}Q(s',a')]$ then $Q' \geq Q$ which is promising. 

This makes $\mathcal{L}(\theta)$ a plausible objective.  

</div>

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

<div style="border: solid 1px #3b8bed; padding: 5px; border-radius: 8px; margin-top: 8px">

I also implemented a `random_batch` method that will be usefull for the DQN `reinforce` method.
</div>

In [9]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        self.memory.append(m)
        if(len(self.memory) > self.max_memory):
            self.memory.pop(0)

    def random_access(self):
        i = np.random.randint(0, len(self.memory))
        return self.memory[i]
    
    def random_batch(self, batch_size):
        # sample a random batch from the samples in `memory`
        # returns 5 ndarrays with shape[0] == batch_size.
        state_shape = self.memory[0][0].shape
        states = np.zeros((batch_size, *state_shape))
        next_states = np.zeros_like(states)
        rewards = np.zeros(batch_size)
        game_overs = np.zeros(batch_size)
        actions = np.zeros(batch_size, dtype=int)
        for i in range(batch_size):
            s, ns, a, r, game_over = self.random_access()
            states[i, :, :, :] = s
            next_states[i, :, :, :] = ns
            rewards[i] = r
            actions[i] = a
            if(game_over):
                game_overs[i] = 1
        return states, next_states, actions, rewards, game_overs

***
The pipeline we will use for training is given below:

In [10]:
def train(agent, env, epoch, prefix='', verbose=1):
    score = 0
    loss = 0
    best_score = -1000

    for e in range(1, epoch+1):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

            
        # Update stats
        score += win-lose
        
        # Save as a mp4
        if e % verbose == 0:
            env.draw(prefix+str(e))
            print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} | Score {:.2f}"
                  .format(e, epoch, loss, win, lose, score/verbose))
            if(score > best_score):
                agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
            score = 0

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [11]:
class DQN(Agent):
    def __init__(self, grid_size, epsilon=0.1, memory_size=100, batch_size=16, n_state=2):
        super(DQN, self).__init__(epsilon=epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        return self.model.predict(np.array([s])).argmax()

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool
        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        # First, sample the batch data
        states, next_states, actions, rewards, game_overs = self.memory.random_batch(self.batch_size)
        
        # Then compute target_q (computations are vectorized)
        qs = self.model.predict(states)
        next_qs = self.model.predict(next_states)
        target_q = qs
        target_qa = rewards + (1 - game_overs)*self.discount*np.max(next_qs, axis=1)
        # Clip the target to avoid exploiding gradients :
        target_qa = np.clip(target_qa, -5, 5)
        target_q[np.arange(self.batch_size), actions] = target_qa
        
        input_states = states
        l = self.model.train_on_batch(input_states, target_q)
        
        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        model = Sequential()
        model.add(Reshape((50,), input_shape=(5,5,2)))
        model.add(Dense(80, input_dim=50, activation='relu'))
        model.add(Dense(30, activation='relu'))
        model.add(Dense(4, activation=None))
        
        model.compile(adam(learning_rate=lr), "mse")
        self.model = model

In [12]:
epochs_train = 50

env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=0.01, epsilon=0.2, memory_size=2000, batch_size=32)
train(agent, env, epochs_train, prefix='fc_train', verbose=10)
HTML(display_videos(f'fc_train{epochs_train}.mp4'))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Epoch 010/050 | Loss 0.0060 | Win/lose count 14.0/6.0 | Score 4.20
Epoch 020/050 | Loss 0.0080 | Win/lose count 5.5/2.0 | Score 4.45
Epoch 030/050 | Loss 0.0017 | Win/lose count 5.0/9.0 | Score 4.30
Epoch 040/050 | Loss 0.0031 | Win/lose count 13.0/9.0 | Score 5.95
Epoch 050/050 | Loss 0.0036 | Win/lose count 8.5/4.0 | Score 4.05


***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [13]:
class DQN_CNN(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        model = Sequential()
        model.add(Conv2D(4, (3, 3), activation='relu', input_shape=(5, 5, self.n_state)))
        #model.add(MaxPooling2D())
        model.add(Conv2D(8, (2, 2), activation='relu'))
        model.add(Reshape((2*2*8,), input_shape=(2, 2, 8)))
        model.add(Dense(4, activation=None))
        
        model.compile(adam(learning_rate=lr), "mse")
        self.model = model

In [14]:
epochs_train = 50
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=0.005, epsilon=0.2, memory_size=2000, batch_size=32)
train(agent, env, epochs_train, prefix='cnn_train', verbose=10)
HTML(display_videos(f'cnn_train{epochs_train}.mp4'))

Epoch 010/050 | Loss 0.0093 | Win/lose count 5.0/5.0 | Score 2.05
Epoch 020/050 | Loss 0.0076 | Win/lose count 18.5/6.0 | Score 4.30
Epoch 030/050 | Loss 0.0041 | Win/lose count 7.0/7.0 | Score 6.90
Epoch 040/050 | Loss 0.0159 | Win/lose count 7.0/4.0 | Score 4.45
Epoch 050/050 | Loss 0.1599 | Win/lose count 8.0/4.0 | Score 4.50


***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [15]:
epochs_test = 40
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.6, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test', verbose=10)
print('\nTest of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test', verbose=10)

Test of the CNN
Win/lose count 1.0/0. Average score (0.8)
Win/lose count 1.5/0. Average score (1.075)
Win/lose count 0.5/0. Average score (1.2)
Win/lose count 2.0/0. Average score (1.4375)
Final score: 1.4375

Test of the FC
Win/lose count 1.5/3.0. Average score (-0.35)
Win/lose count 2.5/1.0. Average score (-0.075)
Win/lose count 1.0/5.0. Average score (-0.26666666666666666)
Win/lose count 2.0/1.0. Average score (-0.3625)
Final score: -0.3625


In [16]:
HTML(display_videos(f'cnn_test{epochs_test}.mp4'))

In [17]:
HTML(display_videos(f'fc_test{epochs_test}.mp4'))

<div style="border: solid 1px #3b8bed; padding: 5px; border-radius: 8px; margin-top: 8px">

The agent loops after 10/20 moves max. There is a clear lack of exploration. The phenomenon can be observed (more or less strongly) at all temperatures.
</div>

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [18]:
def train_explore(agent, env, epoch, decrease_factor=0.9, prefix='', verbose=1):
    # Number of won games
    score = 0
    loss = 0
    best_score = -1000

    for e in range(1, epoch+1):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)
            
        agent.set_epsilon(agent.epsilon * decrease_factor)

        # Update stats
        score += win-lose
        
        if(e % verbose == 0):
            env.draw(prefix+str(e))
            print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose {}/{:.1f} | Score {:.2f}"
                  .format(e, epoch, loss, win, lose, score/verbose))
            if(score > best_score):
                agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
                best_score = score
            score = 0

class EnvironmentExploring(Environment):
    
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        super().__init__(grid_size, max_time, temperature)
        
    def act(self, action, train=False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.create_position(self.x, self.y)
        reward, game_over = self.move(action)
        if(train):
            reward -= self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = 0.1
        state = self.create_state(self.board, self.position, self.malus_position)
        return state, reward, game_over
    
    def reset(self):
        """This function resets the game and returns the initial state"""
        self.reset_board()
        self.create_position()
        self.malus_position = np.zeros((self.grid_size, self.grid_size))
        self.malus_position[self.x, self.y] = 1
        self.t = 0
        
        return self.create_state(self.board, self.position, self.malus_position)

In [19]:
# Training
epochs_train = 100
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.005, epsilon=0.4, memory_size=3000, batch_size=64, n_state=3)
train_explore(agent, env, epochs_train, decrease_factor=0.95, prefix='cnn_train_explore', verbose=5)
HTML(display_videos(f"cnn_train_explore{epochs_train}.mp4"))

Epoch 005/100 | Loss 0.0114 | Win/lose 14.5/18.8 | Score -11.66
Epoch 010/100 | Loss 0.0089 | Win/lose 11.0/18.8 | Score -4.58
Epoch 015/100 | Loss 0.0157 | Win/lose 22.5/19.0 | Score -4.38
Epoch 020/100 | Loss 0.0066 | Win/lose 16.0/18.1 | Score -1.22
Epoch 025/100 | Loss 0.0078 | Win/lose 15.5/20.8 | Score -0.88
Epoch 030/100 | Loss 0.0183 | Win/lose 10.0/18.4 | Score 2.60
Epoch 035/100 | Loss 0.0222 | Win/lose 20.5/16.6 | Score 2.02
Epoch 040/100 | Loss 0.0125 | Win/lose 16.0/15.7 | Score -0.26
Epoch 045/100 | Loss 0.0189 | Win/lose 16.0/18.5 | Score -0.20
Epoch 050/100 | Loss 0.0118 | Win/lose 25.5/11.9 | Score 6.64
Epoch 055/100 | Loss 0.0678 | Win/lose 23.0/16.9 | Score 4.50
Epoch 060/100 | Loss 0.0151 | Win/lose 12.5/14.6 | Score 1.80
Epoch 065/100 | Loss 0.0194 | Win/lose 23.0/13.8 | Score 3.54
Epoch 070/100 | Loss 0.0164 | Win/lose 11.0/21.9 | Score -0.80
Epoch 075/100 | Loss 0.0210 | Win/lose 24.5/13.7 | Score 1.48
Epoch 080/100 | Loss 0.0094 | Win/lose 19.5/20.4 | Score 0.86

In [20]:
# Evaluation
test(agent, env, epochs_test, prefix='cnn_test_explore', verbose=5)
HTML(display_videos(f'cnn_test_explore{epochs_test}.mp4'))

Win/lose count 9.5/3.0. Average score (7.8)
Win/lose count 16.5/3.0. Average score (10.9)
Win/lose count 16.0/0. Average score (10.833333333333334)
Win/lose count 2.5/0. Average score (9.15)
Win/lose count 20.0/1.0. Average score (9.1)
Win/lose count 2.5/0. Average score (8.5)
Win/lose count 5.5/2.0. Average score (8.0)
Win/lose count 2.0/0. Average score (7.7125)
Final score: 7.7125
